1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


__Hit_rate@k__ будем использовать тогда когда ожидаем от покупателя разового действаия - одну покупку (покупка дома, квартиры, машины, отдыха за границей).  
__Precision@k__ будем использовать тогда когда имеем слчай покупки многих вещей одновременно (поход в гастроном).

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Вместо логарифма можно использовать любую другую "медленно и плавно" возрастающую функцию. На пример модуль корня n-й степени. Показателем степени можем отрегулировать веса позиций.  
Логарифмирование или взятие степени - широко известный прием изменения исходного распределения данных.  
Есть функция ошибки: __MeanSquaredLogarithmicError__ loss = square(log(y_true + 1.) - log(y_pred + 1.))

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

__Мера Жаккара.__  Можно исползовать для оценки того насколько хорошо было сделано предсказание покупок пользователя по сравнению с действительными покупками.

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

In [1]:
import numpy as np
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]
recommended = recommended[:8]
recommended

[2, 5, 7, 4, 11, 9, 8, 10]

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [4]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result

In [5]:
def get_NDCG(recommended_list, bought_list):
    up=0.
    down=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                if i==0:
                    up=1
                else:
                    up = up + (i+1)/(np.log(i+1))
        if i==0:
            down=1    
        else:
            down = down+(i+1)/(np.log(i+1))
    result=up/down            
    return result

In [6]:
def reciprocal_rank(recommended_list, bought_list):
    ranks=0.
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)

$pr_at8 =  len([5, 7, 9, 11])/len([2, 5, 7, 4, 11, 9, 8, 10])$     
$pr_at8 = 0.5$

In [7]:
pr_at8 = precision_at_k(recommended, boughted, k=8)
pr_at8

0.5

$rec_at8 = len([5, 7, 11, 9])/len([1, 3, 5, 7, 9, 11])$  
$rec_at8 = 0.6666666666666666$

In [8]:
rec_at8 = recall_at_k(recommended, boughted, k=8)
rec_at8

0.6666666666666666

$[1, 3, 5, 7, 9, 11]$  
$[2, 5, 7, 4, 11, 9, 8, 10]$  
$pr_at1 = 0$  
$pr_at2 = 1/2$  
$pr_at3 = 2/3$  
$pr_at4 = 0$  
$pr_at5 = 3/5$  
$pr_at6 = 4/6$  
$pr_at7 = 0$  
$pr_at8 = 0$  
  
$ap_at1 = (0 + 1/2 + 2/3 + 0 + 3/5 + 4/6 + 0 + 0)/8$  
$ap_at1 = 0.30416666666666664$

In [9]:
ap_at8 = ap_k(recommended, boughted, k=8)
ap_at8

0.30416666666666664

$[1, 3, 5, 7, 9, 11]$  
$[2, 5, 7, 4, 11, 9, 8, 10]$  

$ndcg_at8 = (0 + 1/log(2) + 1/log(3) + 0 + 1/log(5) + 1/log(6) + 0 + 0) / 
(1 + 1/log(2) + 1/log(3) + 1/log(4) + 1/log(5) + 1/log(6) + 1/log(7) + 1/log(8))$  
$ndcg_at8 = 0.5158448585530564$

In [10]:
ndcg_at8 = get_NDCG(recommended, boughted)
ndcg_at8

0.5158448585530564

$[1, 3, 5, 7, 9, 11]$  
$[2, 5, 7, 4, 11, 9, 8, 10]$  

$mrr_at8 = (1/2 + 1/3 + 1/5 + 1/6)/8$  
$mrr_at8 = 0.15$

In [11]:
mrr_at8 = reciprocal_rank(recommended, boughted)
mrr_at8

0.15